In [ ]:
from itertools import product
import numpy as np
from sklearn.metrics import average_precision_score

# 하이퍼파라미터 후보 설정
lr_list = [1e-4, 1e-3, 1e-2]
weight_decay_list = [1e-4, 1e-3, 1e-2]
drop_rate_list = [0.0, 0.1, 0.2, 0.3]

# 모든 조합 생성
param_combinations = list(product(lr_list, weight_decay_list, drop_rate_list))

# 결과 저장
results = []

kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for params in param_combinations:
    lr, weight_decay, drop_rate = params
    print(f"Testing params: lr={lr}, weight_decay={weight_decay}, drop_rate={drop_rate}")
    
    acc_scores, precision_scores, recall_scores, f1_scores, roc_auc_scores, pr_auc_scores = [], [], [], [], [], []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f'Fold {fold + 1}/{k_folds}')
        
        # kfold index를 이용해서 train data와 val data의 분리
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

        # 사용자 정의 데이터셋
        train_dataset = CustomDataset(X_train_fold, y_train_fold)
        val_dataset = CustomDataset(X_val_fold, y_val_fold)

        # 데이터로더 생성
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        # 모델 생성
        model = MLP(nBits, drop_rate).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = nn.BCELoss().to(device)
        
        # Early stopping 초기화
        early_stopping = EarlyStopping(patience=patience, delta=0.001)

        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs.to(device))
                loss = criterion(outputs, labels.to(device).float().unsqueeze(1))  # labels 크기 맞춤
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            # Validation step
            model.eval()
            val_loss = 0.0
            val_preds = []
            val_targets = []
            
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs = inputs.to(device)
                    labels = labels.to(device).float().unsqueeze(1)  # labels 크기 맞춤
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    
                    # 예측값 수집
                    val_preds.extend(outputs.cpu().numpy().flatten())
                    val_targets.extend(labels.cpu().numpy().flatten())
            
            val_loss /= len(val_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss}")

            # Early Stopping 체크
            early_stopping(val_loss)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch+1}")
                break

        # 성능 지표 계산
        y_true = np.array(val_targets).flatten()
        y_pred = np.array(val_preds).flatten()

        acc_scores.append(accuracy_score(y_true, (y_pred > 0.5).astype(int)))
        precision_scores.append(precision_score(y_true, (y_pred > 0.5).astype(int)))
        recall_scores.append(recall_score(y_true, (y_pred > 0.5).astype(int)))
        f1_scores.append(f1_score(y_true, (y_pred > 0.5).astype(int)))
        roc_auc_scores.append(roc_auc_score(y_true, y_pred))
        pr_auc_scores.append(average_precision_score(y_true, y_pred))

    # Cross-validation 평균 점수 계산
    acc_mean = np.mean(acc_scores)
    precision_mean = np.mean(precision_scores)
    recall_mean = np.mean(recall_scores)
    f1_mean = np.mean(f1_scores)
    roc_auc_mean = np.mean(roc_auc_scores)
    pr_auc_mean = np.mean(pr_auc_scores)

    # 결과 저장
    results.append({
        'lr': lr,
        'weight_decay': weight_decay,
        'drop_rate': drop_rate,
        'acc_mean': acc_mean,
        'precision_mean': precision_mean,
        'recall_mean': recall_mean,
        'f1_mean': f1_mean,
        'roc_auc_mean': roc_auc_mean,
        'pr_auc_mean': pr_auc_mean,
    })

# ROC AUC를 기준으로 최적의 하이퍼파라미터 선택
best_result = max(results, key=lambda x: x['roc_auc_mean'])

print(f"Best params based on ROC AUC: {best_result}")